In [385]:
import sqlite3
import pandas as pd

In [416]:
# Created connection.
database_path = r"database.sqlite"

conn = sqlite3.connect(database_path)

In [417]:
# Creating dataframes from the tbales

df_country = pd.read_sql_query("SELECT * FROM country", conn)
df_league = pd.read_sql_query("SELECT * FROM league", conn)
df_match = pd.read_sql_query("SELECT * FROM match", conn)
df_player = pd.read_sql_query("SELECT * FROM player", conn)
df_player_attributes = pd.read_sql_query("SELECT * FROM player_attributes", conn)
df_team = pd.read_sql_query("SELECT * FROM team", conn)
df_team_attributes = pd.read_sql_query("SELECT * FROM team_attributes", conn)
df_sqlite_sequence = pd.read_sql_query("SELECT * FROM sqlite_sequence", conn)


In [418]:
# Deleting Duplicate values from the table

df_country = df_country.drop_duplicates()
df_league = df_league.drop_duplicates()
df_match = df_match.drop_duplicates()
df_player = df_player.drop_duplicates()
df_player_attributes = df_player_attributes.drop_duplicates()
df_team = df_team.drop_duplicates()
df_team_attributes = df_team_attributes.drop_duplicates()
df_sqlite_sequence = df_sqlite_sequence.drop_duplicates()

In [419]:
# Concate all the tables in one
# Note: removed and merged by analyzed the all table relationship by zeel. so, before making change make sure to communicate eith zeel

In [421]:
df_country.shape,df_league.shape

((11, 2), (11, 3))

In [422]:
# merging country table
df_merged_all_table = pd.merge(left=df_league, right=df_country, how='outer', left_on='country_id', right_on='id')
df_merged_all_table = df_merged_all_table.drop(["id_x", "country_id"], axis=1)
df_merged_all_table.columns.values[0] = "league_name"
df_merged_all_table.columns.values[1] = "league_id"
df_merged_all_table.columns.values[2] = "country_name"

In [424]:
df_merged_all_table

,league_name,league_id,country_name
0,Belgium Jupiler League,1,Belgium
1,England Premier League,1729,England
2,France Ligue 1,4769,France
3,Germany 1. Bundesliga,7809,Germany
4,Italy Serie A,10257,Italy
5,Netherlands Eredivisie,13274,Netherlands
6,Poland Ekstraklasa,15722,Poland
7,Portugal Liga ZON Sagres,17642,Portugal
8,Scotland Premier League,19694,Scotland
9,Spain LIGA BBVA,21518,Spain


In [425]:
pd.merge(left=df_team, right=df_team_attributes, how='outer', left_on=['team_api_id'], right_on=['team_api_id']).shape

(1469, 29)

In [431]:
df_merged_all_table.head()

,league_name,league_id,country_name
0,Belgium Jupiler League,1,Belgium
1,England Premier League,1729,England
2,France Ligue 1,4769,France
3,Germany 1. Bundesliga,7809,Germany
4,Italy Serie A,10257,Italy


In [432]:
df_merged_team_table.head()

,team_api_id,team_fifa_api_id,team_long_name,team_short_name,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,9987,673.0,KRC Genk,GEN,2010-02-22 00:00:00,45.0,Balanced,NaN,Little,45.0,...,60.0,Normal,Organised,70.0,High,65.0,Press,70.0,Wide,Cover
1,9987,673.0,KRC Genk,GEN,2011-02-22 00:00:00,66.0,Balanced,NaN,Little,52.0,...,51.0,Normal,Organised,48.0,Medium,47.0,Press,54.0,Normal,Offside Trap
2,9987,673.0,KRC Genk,GEN,2012-02-22 00:00:00,53.0,Balanced,NaN,Little,55.0,...,56.0,Normal,Organised,47.0,Medium,45.0,Press,55.0,Normal,Cover
3,9987,673.0,KRC Genk,GEN,2013-09-20 00:00:00,58.0,Balanced,NaN,Little,38.0,...,56.0,Normal,Organised,47.0,Medium,45.0,Press,55.0,Normal,Cover
4,9987,673.0,KRC Genk,GEN,2014-09-19 00:00:00,58.0,Balanced,52.0,Normal,38.0,...,56.0,Normal,Organised,47.0,Medium,45.0,Press,55.0,Normal,Cover


In [433]:
df_merged_all_table.head()

,league_name,league_id,country_name
0,Belgium Jupiler League,1,Belgium
1,England Premier League,1729,England
2,France Ligue 1,4769,France
3,Germany 1. Bundesliga,7809,Germany
4,Italy Serie A,10257,Italy


In [434]:
df_match.merge(df_merged_all_table, how='outer', left_on='league_id', right_on='league_id').shape

(25979, 117)

In [435]:
# merging league table
df_merged_all_table = df_match.merge(df_merged_all_table, how='outer', left_on='league_id', right_on='league_id')
df_merged_all_table = df_merged_all_table.drop(["league_id", "country_id"], axis=1)
df_merged_all_table.columns.values[0] = "match_id" 
df_merged_all_table = df_merged_all_table.drop(["match_id"], axis=1)

In [436]:
df_merged_all_table.shape

(25979, 114)

In [ ]:
# merging merged team table
df_merged_all_table = df_merged_all_table.merge(df_merged_team_table, how='outer', left_on=['home_team_api_id'], right_on=['team_api_id'])
df_merged_all_table = df_merged_all_table.drop(["date_y"], axis=1)
df_merged_all_table.columns.values[2] = "date" 
df_merged_all_table = df_merged_all_table.merge(df_merged_team_table, how='outer', left_on=['away_team_api_id'], right_on=['team_api_id'])
df_merged_all_table.columns.values[2] = "date" 

for i in range(26):
    df_merged_all_table = df_merged_all_table.drop([df_merged_all_table.columns[-1]], axis=1)

for i in range(26):
    df_merged_all_table.columns.values[-1 - i] = df_merged_all_table.columns[-1 - i].split("_x")[0]

In [437]:
# merging player_attribute table with player table
df_merged_player_table = pd.merge(left=df_player, right=df_player_attributes, how='outer', left_on=['player_api_id'], right_on=['player_api_id'])
df_merged_player_table = df_merged_player_table.drop(["id_x", "id_y", "player_fifa_api_id_y"], axis=1)
df_merged_player_table.columns.values[2] = "player_fifa_api_id" 

df_merged_player_table.drop_duplicates() 

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2016-02-18 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-11-19 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-09-21 00:00:00,62.0,66.0,right,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-03-20 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2007-02-22 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2009-08-30 00:00:00,83.0,85.0,right,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2009-02-22 00:00:00,78.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2008-08-30 00:00:00,77.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2007-08-30 00:00:00,78.0,81.0,right,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [442]:
# droping columns with contains id (Converting n3 form to n1 form)
# df_merged_all_table = df_merged_all_table.drop(["team_fifa_api_id", "team_api_id", "match_api_id"], axis=1)

In [443]:
df_merged_all_table.shape

(25979, 114)

In [444]:
df_merged_all_table.drop_duplicates(inplace=True)

In [445]:
df_merged_all_table.shape

(25979, 114)

In [446]:
print(df_merged_all_table.columns)

Index(['season', 'stage', 'date', 'match_api_id', 'home_team_api_id',
       'away_team_api_id', 'home_team_goal', 'away_team_goal',
       'home_player_X1', 'home_player_X2',
       ...
       'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'league_name',
       'country_name'],
      dtype='object', length=114)


In [447]:
# combining total goal of a respected team

home_total_goal = pd.read_sql_query("select home_team_api_id, sum(home_team_goal) as home_total_goal from match group by home_team_api_id", conn)
away_total_goal = pd.read_sql_query("select away_team_api_id, sum(away_team_goal) as away_total_goal from match group by away_team_api_id", conn)

In [448]:
home_total_goal.shape

(299, 2)

In [449]:
home_total_goal.head()

,home_team_api_id,home_total_goal
0,1601,148
1,1773,74
2,1957,176
3,2033,70
4,2182,215


In [453]:
pd.merge(df_merged_all_table, away_total_goal, on='away_team_api_id').shape

(25979, 115)

In [454]:
# merging both the home and away total goal to the match table

df_merged_all_table = pd.merge(df_merged_all_table, home_total_goal, on='home_team_api_id')
df_merged_all_table = pd.merge(df_merged_all_table, away_total_goal, on='away_team_api_id')

In [455]:
df_merged_all_table.shape

(25979, 116)

In [395]:
df_merged_player_table.head()

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2016-02-18 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-11-19 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-09-21 00:00:00,62.0,66.0,right,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-03-20 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2007-02-22 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [456]:
# getting overall rating of player

player_overall_rating = pd.read_sql_query("select * from (select player_api_id , max(overall_rating) as overall_rating from Player_Attributes group by player_api_id) as p where p.overall_rating is not null", conn)


In [457]:
player_overall_rating

,player_api_id,overall_rating
0,2625,63
1,2752,72
2,2768,75
3,2770,74
4,2790,73
...,...,...
11055,744907,53
11056,746419,59
11057,748432,58
11058,750435,60


In [398]:
print(*df_merged_all_table.columns)

season stage date home_team_api_id away_team_api_id home_team_goal away_team_goal home_player_X1 home_player_X2 home_player_X3 home_player_X4 home_player_X5 home_player_X6 home_player_X7 home_player_X8 home_player_X9 home_player_X10 home_player_X11 away_player_X1 away_player_X2 away_player_X3 away_player_X4 away_player_X5 away_player_X6 away_player_X7 away_player_X8 away_player_X9 away_player_X10 away_player_X11 home_player_Y1 home_player_Y2 home_player_Y3 home_player_Y4 home_player_Y5 home_player_Y6 home_player_Y7 home_player_Y8 home_player_Y9 home_player_Y10 home_player_Y11 away_player_Y1 away_player_Y2 away_player_Y3 away_player_Y4 away_player_Y5 away_player_Y6 away_player_Y7 away_player_Y8 away_player_Y9 away_player_Y10 away_player_Y11 home_player_1 home_player_2 home_player_3 home_player_4 home_player_5 home_player_6 home_player_7 home_player_8 home_player_9 home_player_10 home_player_11 away_player_1 away_player_2 away_player_3 away_player_4 away_player_5 away_player_6 away_playe

In [458]:
df_merged_all_table_copy = df_merged_all_table.copy()

In [459]:
for i in range(1,12):
    df_merged_all_table = df_merged_all_table.merge(player_overall_rating, how='left', left_on='home_player_' + str(i), right_on='player_api_id')
    df_merged_all_table = df_merged_all_table.drop(['home_player_' + str(i), "player_api_id"], axis=1)
    df_merged_all_table.columns.values[-1] = 'home_player_' + str(i) 

for i in range(1,12):
    df_merged_all_table = df_merged_all_table.merge(player_overall_rating, how='left', left_on='away_player_' + str(i), right_on='player_api_id')
    df_merged_all_table = df_merged_all_table.drop(['away_player_' + str(i), "player_api_id"], axis=1)
    df_merged_all_table.columns.values[-1] = 'away_player_' + str(i) 

df_merged_all_table = df_merged_all_table.drop([], axis=1)

In [461]:
df_merged_all_table.sample(10)

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
19085,2011/2012,29,2012-05-05 00:00:00,1043055,9772,9771,1,0,1.0,2.0,...,NaN,65.0,70.0,75.0,66.0,60.0,65.0,68.0,NaN,73.0
188,2011/2012,22,2012-01-25 00:00:00,1032866,9989,9984,2,2,1.0,1.0,...,68.0,71.0,71.0,81.0,64.0,68.0,70.0,70.0,69.0,65.0
6638,2011/2012,6,2011-09-17 00:00:00,1019426,8689,9853,3,0,1.0,2.0,...,76.0,78.0,78.0,79.0,79.0,77.0,76.0,84.0,79.0,73.0
16736,2015/2016,23,2016-02-19 00:00:00,1994793,1957,8025,2,1,1.0,2.0,...,64.0,72.0,66.0,66.0,64.0,65.0,64.0,64.0,66.0,67.0
10477,2014/2015,19,2015-01-18 00:00:00,1786158,8543,9875,0,1,1.0,2.0,...,81.0,80.0,83.0,73.0,79.0,77.0,81.0,78.0,82.0,86.0
6310,2009/2010,15,2009-11-29 00:00:00,654174,8481,9827,0,3,1.0,2.0,...,80.0,79.0,84.0,81.0,81.0,79.0,81.0,81.0,77.0,82.0
8850,2013/2014,24,2014-03-09 00:00:00,1479209,9905,8177,1,1,1.0,2.0,...,76.0,77.0,79.0,70.0,76.0,71.0,73.0,78.0,75.0,79.0
21967,2013/2014,25,2014-02-23 00:00:00,1506057,8371,9906,3,0,1.0,2.0,...,81.0,82.0,86.0,82.0,81.0,80.0,82.0,88.0,89.0,86.0
21628,2010/2011,18,2011-01-09 00:00:00,875797,8696,9869,1,1,1.0,2.0,...,69.0,73.0,78.0,75.0,77.0,74.0,74.0,76.0,79.0,76.0
3125,2009/2010,23,2010-01-26 00:00:00,658957,8586,9879,2,0,1.0,2.0,...,78.0,83.0,83.0,75.0,72.0,72.0,77.0,81.0,77.0,80.0


In [400]:
print(*df_merged_all_table.columns)

season stage date home_team_api_id away_team_api_id home_team_goal away_team_goal home_player_X1 home_player_X2 home_player_X3 home_player_X4 home_player_X5 home_player_X6 home_player_X7 home_player_X8 home_player_X9 home_player_X10 home_player_X11 away_player_X1 away_player_X2 away_player_X3 away_player_X4 away_player_X5 away_player_X6 away_player_X7 away_player_X8 away_player_X9 away_player_X10 away_player_X11 home_player_Y1 home_player_Y2 home_player_Y3 home_player_Y4 home_player_Y5 home_player_Y6 home_player_Y7 home_player_Y8 home_player_Y9 home_player_Y10 home_player_Y11 away_player_Y1 away_player_Y2 away_player_Y3 away_player_Y4 away_player_Y5 away_player_Y6 away_player_Y7 away_player_Y8 away_player_Y9 away_player_Y10 away_player_Y11 goal shoton shotoff foulcommit card cross corner possession B365H B365D B365A BWH BWD BWA IWH IWD IWA LBH LBD LBA PSH PSD PSA WHH WHD WHA SJH SJD SJA VCH VCD VCA GBH GBD GBA BSH BSD BSA league_name country_name team_long_name team_short_name buildUpP

In [401]:
temp_list = "season stage date home_team_api_id away_team_api_id home_player_X1 home_player_X2 home_player_X3 home_player_X4 home_player_X5 home_player_X6 home_player_X7 home_player_X8 home_player_X9 home_player_X10 home_player_X11 away_player_X1 away_player_X2 away_player_X3 away_player_X4 away_player_X5 away_player_X6 away_player_X7 away_player_X8 away_player_X9 away_player_X10 away_player_X11 home_player_Y1 home_player_Y2 home_player_Y3 home_player_Y4 home_player_Y5 home_player_Y6 home_player_Y7 home_player_Y8 home_player_Y9 home_player_Y10 home_player_Y11 away_player_Y1 away_player_Y2 away_player_Y3 away_player_Y4 away_player_Y5 away_player_Y6 away_player_Y7 away_player_Y8 away_player_Y9 away_player_Y10 away_player_Y11".split(" ")

In [402]:
temp_list

['season',
 'stage',
 'date',
 'home_team_api_id',
 'away_team_api_id',
 'home_player_X1',
 'home_player_X2',
 'home_player_X3',
 'home_player_X4',
 'home_player_X5',
 'home_player_X6',
 'home_player_X7',
 'home_player_X8',
 'home_player_X9',
 'home_player_X10',
 'home_player_X11',
 'away_player_X1',
 'away_player_X2',
 'away_player_X3',
 'away_player_X4',
 'away_player_X5',
 'away_player_X6',
 'away_player_X7',
 'away_player_X8',
 'away_player_X9',
 'away_player_X10',
 'away_player_X11',
 'home_player_Y1',
 'home_player_Y2',
 'home_player_Y3',
 'home_player_Y4',
 'home_player_Y5',
 'home_player_Y6',
 'home_player_Y7',
 'home_player_Y8',
 'home_player_Y9',
 'home_player_Y10',
 'home_player_Y11',
 'away_player_Y1',
 'away_player_Y2',
 'away_player_Y3',
 'away_player_Y4',
 'away_player_Y5',
 'away_player_Y6',
 'away_player_Y7',
 'away_player_Y8',
 'away_player_Y9',
 'away_player_Y10',
 'away_player_Y11']

In [403]:
# droping columns with contains id (Converting n3 form to n1 form)
# df_merged_all_table = df_merged_all_table.drop(columns=["team_fifa_api_id", "team_api_id", "match_api_id"])

In [404]:
df_merged_all_table.columns

Index(['season', 'stage', 'date', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_X1', 'home_player_X2',
       'home_player_X3',
       ...
       'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5',
       'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9',
       'away_player_10', 'away_player_11'],
      dtype='object', length=138)

In [405]:
df_merged_all_table = df_merged_all_table.drop(columns= temp_list)

In [406]:
df_merged_all_table.columns

Index(['home_team_goal', 'away_team_goal', 'goal', 'shoton', 'shotoff',
       'foulcommit', 'card', 'cross', 'corner', 'possession', 'B365H', 'B365D',
       'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA',
       'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH',
       'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'league_name',
       'country_name', 'team_long_name', 'team_short_name', 'buildUpPlaySpeed',
       'buildUpPlaySpeedClass', 'buildUpPlayDribbling',
       'buildUpPlayDribblingClass', 'buildUpPlayPassing',
       'buildUpPlayPassingClass', 'buildUpPlayPositioningClass',
       'chanceCreationPassing', 'chanceCreationPassingClass',
       'chanceCreationCrossing', 'chanceCreationCrossingClass',
       'chanceCreationShooting', 'chanceCreationShootingClass',
       'chanceCreationPositioningClass', 'defencePressure',
       'defencePressureClass', 'defenceAggression', 'defenceAggressionClass',
       'defenceTeamWidth', 

In [407]:
# droping columns with contains id (Converting n3 form to n1 form)
df_merged_all_table = df_merged_all_table.drop(['league_name',
       'country_name', 'team_long_name', 'team_short_name', 'buildUpPlaySpeed',
       'buildUpPlaySpeedClass', 'buildUpPlayDribbling',
       'buildUpPlayDribblingClass', 'buildUpPlayPassing',
       'buildUpPlayPassingClass', 'buildUpPlayPositioningClass',
       'chanceCreationPassing', 'chanceCreationPassingClass',
       'chanceCreationCrossing', 'chanceCreationCrossingClass',
       'chanceCreationShooting', 'chanceCreationShootingClass',
       'chanceCreationPositioningClass', 'defencePressure',
       'defencePressureClass', 'defenceAggression', 'defenceAggressionClass',
       'defenceTeamWidth', 'defenceTeamWidthClass', 'defenceDefenderLineClass', 'goal', 'shoton', 'shotoff',
       'foulcommit', 'card', 'cross', 'corner', 'possession'], axis=1)

df_merged_all_table.columns

Index(['home_team_goal', 'away_team_goal', 'B365H', 'B365D', 'B365A', 'BWH',
       'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD',
       'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA',
       'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'home_total_goal',
       'away_total_goal', 'home_player_1', 'home_player_2', 'home_player_3',
       'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7',
       'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11'],
      dtype='object')

In [408]:
df_merged_all_table.columns

Index(['home_team_goal', 'away_team_goal', 'B365H', 'B365D', 'B365A', 'BWH',
       'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD',
       'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA',
       'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'home_total_goal',
       'away_total_goal', 'home_player_1', 'home_player_2', 'home_player_3',
       'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7',
       'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11'],
      dtype='object')

In [409]:
df_merged_all_table.sample()

,home_team_goal,away_team_goal,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
18918,3,2,8.5,4.75,1.44,8.25,4.2,1.42,7.0,4.0,...,83.0,80.0,82.0,83.0,86.0,82.0,86.0,88.0,88.0,85.0


In [462]:
df_merged_all_table.shape

(25979, 116)

In [463]:
# generating target variable
def get_match_label(match):
    home_goals = match['home_team_goal']
    away_goals = match['away_team_goal']
    #Identify match label
    if home_goals > away_goals:
        return "HomeWin"
    elif home_goals == away_goals:
        return "Draw"
    elif home_goals < away_goals:
        return "AwayWin"

df_merged_all_table['target'] = df_merged_all_table.apply(get_match_label, axis=1)

In [464]:
df_merged_all_table.shape

(25979, 117)

In [465]:
# droping columns with contains id (Converting n3 form to n1 form)
df_merged_all_table = df_merged_all_table.drop(['home_team_goal',
       'away_team_goal'], axis=1)

df_merged_all_table.columns

Index(['season', 'stage', 'date', 'match_api_id', 'home_team_api_id',
       'away_team_api_id', 'home_player_X1', 'home_player_X2',
       'home_player_X3', 'home_player_X4',
       ...
       'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6',
       'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10',
       'away_player_11', 'target'],
      dtype='object', length=115)

In [466]:
df_merged_all_table.shape

(25979, 115)

In [415]:
# Converting dataframe into csv for modeling

df_merged_all_table.to_pickle('merged_all_table.csv' + ".pkl", compression="bz2")
df_merged_all_table.to_csv( 'merged_all_table.csv',index=False)